In [5]:
%matplotlib widget

from util import get_path, get_dates_datetime, get_dirname
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
    from_nx_to_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign, realign_final
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy, copy
from collections import Counter
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import (
    dic_to_sparse,
    from_sparse_to_graph,
    generate_nx_graph,
    prune_graph,
    from_nx_to_tab,
    from_nx_to_tab_matlab,
    sparse_to_doc,
    connections_pixel_list_to_tab,
    transform_list,
    clean_degree_4,
    get_degree3_nodes,
)
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration
import open3d as o3d
from cycpd import rigid_registration
import sys
i = 146
plate = 29
thresh = 100000
# directory = "/projects/0/einf914/data/"
from directory import directory, run_parallel, find_state

dates_datetime = get_dates_datetime(directory,plate)
dates_datetime.sort()
dates_datetime_chosen = dates_datetime[i : i + 2]
print("========")
print(f"Matching plate {plate} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen
kernel = np.ones((5,5),np.uint8)
itera = 1
dilateds = []
skels2 = []
skels = []
skel_docs = []
for date in dates:
    print (date)
    directory_name = get_dirname(date, plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap + "/Analysis/skeleton_pruned.mat")
    skel_info2 = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info["skeleton"]
    skel2 = skel_info2['skeleton']
    skels2.append(cv2.dilate(skel2.astype(np.uint8),kernel,iterations = itera))
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)



Matching plate 29 at dates [datetime.datetime(2020, 9, 22, 8, 28), datetime.datetime(2020, 9, 22, 13, 37)]
2020-09-22 08:28:00
2020-09-22 13:37:00


In [3]:
plt.close('all')
start = 0
finish = start +1
for i in range(start,finish+1):
    plot_t_tp1([],[],None,None,skels2[i],skels2[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
isnan = True
for order in [(0,1),(1,0)]:
    skeleton1, skeleton2 = skel_docs[order[0]], skel_docs[order[1]]
    skelet_pos = np.array(list(skeleton1.keys()))
    samples = np.random.choice(skelet_pos.shape[0], len(skeleton2.keys()) // 100)
    X = np.transpose(skelet_pos[samples, :])
    skelet_pos = np.array(list(skeleton2.keys()))
    samples = np.random.choice(skelet_pos.shape[0], len(skeleton2.keys()) // 100)
    Y = np.transpose(skelet_pos[samples, :])
    reg = rigid_registration(
        **{
            "X": np.transpose(X.astype(float)),
            "Y": np.transpose(Y.astype(float)),
            "scale": False,'tolerance' : 1e-5, 'w' : 1e-5
        }
    )
    out = reg.register()
    Rfound = reg.R[0:2, 0:2]
    tfound = np.dot(Rfound, reg.t[0:2])
    if order == (0,1):
        t_init = -tfound
        Rot_init = Rfound
    else:
        Rot_init,t_init = np.linalg.inv(Rfound), np.dot(np.linalg.inv(Rfound),tfound)
    sigma2 = reg.sigma2
    if sigma2>=thresh:
        print("========")
        print(f"Failed to match plate {plate} at dates {dates_datetime_chosen}")
        print("========")
        continue
    isnan = np.isnan(tfound[0])
    if isnan:
        continue
#     X = np.transpose(
#         np.array([pos1[node] for node in pruned1 if pruned1.degree(node) == 3])
#     )
#     Y = np.transpose(
#         np.array([pos2[node] for node in pruned2 if pruned2.degree(node) == 3])
#     )
    skeleton1, skeleton2 = skel_docs[0], skel_docs[1]
    X = np.transpose(
        np.array(get_degree3_nodes(skeleton1))
    )
    Y = np.transpose(
        np.array(get_degree3_nodes(skeleton2))
    )
    # fig=plt.figure(figsize=(10,9))
    # ax = fig.add_subplot(111)
    # ax.scatter(X[0,:],X[1,:])
    # ax.scatter(Y[0,:],Y[1,:])
#     Xex = np.transpose(np.transpose(np.dot(Rot_init, X)) + t_init)
    # fig=plt.figure(figsize=(10,9))
    # ax = fig.add_subplot(111)
    # ax.scatter(Xex[0,:],Xex[1,:])
    # ax.scatter(Y[0,:],Y[1,:])
    X = np.insert(X, 2, values=0, axis=0)
    Y = np.insert(Y, 2, values=0, axis=0)
    print(X.shape, Y.shape)
    vectorX = o3d.utility.Vector3dVector(np.transpose(X))
    vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
    source = o3d.geometry.PointCloud(vectorX)
    target = o3d.geometry.PointCloud(vectorY)
    threshold = 200
    trans_init = np.asarray(
        [
            [Rot_init[0, 0], Rot_init[0, 1], 0, t_init[0]],
            [Rot_init[1, 0], Rot_init[1, 1], 0, t_init[1]],
            [0, 0, 1, 0],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )
    reg_p2p = o3d.registration.registration_icp(
        source,
        target,
        threshold,
        trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
    )
    print(reg_p2p)
    Rfound = reg_p2p.transformation[0:2, 0:2]
    tfound = reg_p2p.transformation[0:2, 3]
    print(Rfound, tfound)
    X, Y = X[0:2, :], Y[0:2, :]
    Yrep = np.transpose(np.transpose(np.dot(Rfound, X)) + tfound)
    # fig=plt.figure(figsize=(10,9))
    # ax = fig.add_subplot(111)
    # ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    # ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
    break

if not isnan:    
    sio.savemat(path_snap + "/Analysis/transform.mat", {"R": Rfound, "t": tfound})
else :    
    sio.savemat(path_snap + "/Analysis/transform_corrupt.mat", {"R": np.array([[1,0],[0,1]]), "t": np.array([0,0])})

Iteration:1
ML:-40361.878; 	ML change (error): 40361.878; 	Sigma^2:90815506.080; 	Sigma^2 change:107693528.950
[                                                                        ]
Iteration:2
ML:-10201.043; 	ML change (error): 30160.836; 	Sigma^2:55522936.168; 	Sigma^2 change:35292569.911
[=                                                                       ]
Iteration:3
ML:  3924.231; 	ML change (error): 14125.273; 	Sigma^2:38588612.265; 	Sigma^2 change:16934323.903
[==                                                                      ]
Iteration:4
ML: 12438.889; 	ML change (error):  8514.659; 	Sigma^2:28778667.912; 	Sigma^2 change:9809944.353
[==                                                                      ]
Iteration:5
ML: 18336.358; 	ML change (error):  5897.469; 	Sigma^2:22404582.880; 	Sigma^2 change:6374085.032
[===                                                                     ]
Iteration:6
ML: 22812.173; 	ML change (error):  4475.814; 	Sigma^2:17951563.

KeyboardInterrupt: 

In [18]:
skeleton1, skeleton2 = skel_docs[0], skel_docs[1]
skelet_pos = np.array(list(skeleton1.keys()))
samples = np.random.choice(skelet_pos.shape[0], len(skeleton2.keys()) // 100)
X = np.transpose(skelet_pos[samples, :])
skelet_pos = np.array(list(skeleton2.keys()))
samples = np.random.choice(skelet_pos.shape[0], len(skeleton2.keys()) // 100)
Y = np.transpose(skelet_pos[samples, :])
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
R,t = Rfound, tfound
# R,t = np.linalg.inv(Rfound), np.dot(np.linalg.inv(Rfound),tfound)
Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
R,t = transform['R'],transform['t']

In [18]:
tfound[0]

1.09463192611742

In [15]:
R,t

(array([[ 9.99999979e-01,  2.06116401e-04],
        [-2.06116401e-04,  9.99999979e-01]]),
 array([[  1.09463193, -22.7894256 ]]))

In [16]:
dates_datetime_chosen

[datetime.datetime(2020, 7, 19, 5, 25), datetime.datetime(2020, 7, 19, 9, 25)]

In [8]:
%matplotlib widget

Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime

In [15]:
dates_datetime_chosen[12],dates_datetime_chosen[13]

(datetime.datetime(2020, 7, 16, 11, 34), datetime.datetime(2020, 7, 18, 9, 25))

In [24]:
import pandas as pd
import networkx as nx
import numpy as np
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_nx_to_tab
from node_id import whole_movement_identification
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from sparse_util import dilate, zhangSuen
from scipy.optimize import minimize
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
import open3d as o3d


def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = np.round(np.transpose(np.dot(Rot,np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int)
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)

def realign(skeleton1,nx_graphB,posB,convergence_threshold,window=500,maxdist=50,save=''):
    converged=False
    nx_graphA,posA=generate_nx_graph_from_skeleton(skeleton1) 
    t0=np.array([0,0])
    R0=np.identity(2)
    while not converged:
        listeA,listeB = find_common_group_nodes(nx_graphA,nx_graphB,posA,posB,R0,t0,maxdist=maxdist,window=window)
        H=np.dot(np.transpose(np.array(listeA)-np.mean(listeA,axis=0)),np.array(listeB)-np.mean(listeB,axis=0))
        U,S,V=np.linalg.svd(H)
        R=np.dot(V,np.transpose(U))
        t=np.mean(listeB,axis=0)-np.dot(R,np.mean(listeA,axis=0))
        print("number_common_nodes_found :",len(listeA))
        if np.linalg.norm(t)<=convergence_threshold:
            converged=True
        R0=np.dot(R,R0)
        t0=t+t0
    skeleton_transformed=transform_skeleton(skeleton1,R0,t0)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    if len(save)>=0:
        from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(save+'_raw_aligned_skeleton.csv')
        np.savetxt(save+'rot.txt',R0)
        np.savetxt(save+'trans.txt',t0)
    print("R0=",R0,'t0=',t0)
    return(skeleton_transformed)



def find_common_group_nodes(Sa,Sb,degree3_nodesa,degree3_nodesb,posa,posb,R0,t0,window=500,maxdist=50):
    common_nodes_a = []
    common_nodes_b = []
    common_centroida = []
    common_centroidb = []
    t=time()
    posarottrans = {key : np.round(np.transpose(np.dot(R0,np.transpose(np.array(posa[key])))+t0)).astype(np.int) for key in degree3_nodesa}
#     print("rotating translating",time()-t)
    for node in degree3_nodesa:
        t=time()
        posanchor=posarottrans[node]
        potential_surroundinga=Sa[posanchor[0]-2*window:posanchor[0]+2*window,posanchor[1]-2*window:posanchor[1]+2*window]
        potential_surroundingb=Sb[posanchor[0]-2*window:posanchor[0]+2*window,posanchor[1]-2*window:posanchor[1]+2*window]
#         print("candidates",len(potential_surroundinga.data))
#         print("finding_potential_surrounding",time()-t)
        t=time()
        surrounding_nodesa=[node for node in potential_surroundinga.data if 
                            (posanchor[0]-window<posarottrans[int(node)][0]<posanchor[0]+window and posanchor[1]-window<posarottrans[int(node)][1]<posanchor[1]+window 
                             )]
        surrounding_nodesb=[node for node in potential_surroundingb.data if 
                    (posanchor[0]-window<posb[int(node)][0]<posanchor[0]+window and posanchor[1]-window<posb[int(node)][1]<posanchor[1]+window 
                    )]
#         print("finding_surrounding",time()-t)
        t=time()
        if len(surrounding_nodesa)==len(surrounding_nodesb):
            possurroundinga=[posarottrans[node] for node in surrounding_nodesa]
            possurroundingb=[posb[node] for node in surrounding_nodesb]
            centroida= np.mean(possurroundinga,axis=0)
            centroidb= np.mean(possurroundingb,axis=0)
            if np.linalg.norm(centroida-centroidb)<=maxdist:
                common_centroida.append(centroida)
                common_centroidb.append(centroidb)
    return(common_centroida,common_centroidb)

def realign2(skeleton1,skeleton2,convergence_threshold,window=500,maxdist=50,save=''):
    converged=False
    tim=time()
    nx_graphA,posA=generate_nx_graph_from_skeleton(skeleton1)
    nx_graphB,posB=generate_nx_graph_from_skeleton(skeleton2)
    print("generate_nx_graph_from_skeleton, t=",tim-time())
    tim=time()
    t0=np.array([0,0])
    R0=np.identity(2)
    degree3_nodesa = [node for node in nx_graphA if nx_graphA.degree(node)==3]
    degree3_nodesb = [node for node in nx_graphB if nx_graphB.degree(node)==3]
    print("lennodes=",len(degree3_nodesa))
    Sa=sparse.csr_matrix((26296, 49559))
    Sb=sparse.csr_matrix((26296, 49559))
    for node in degree3_nodesa:
        Sa[posA[node][0],posA[node][1]]=node
    for node in degree3_nodesb:
        Sb[posB[node][0],posB[node][1]]=node
    while not converged:
        listeA,listeB = find_common_group_nodes(Sa,Sb,degree3_nodesa,degree3_nodesb,posA,posB,R0,t0,maxdist=maxdist,window=window)
        H=np.dot(np.transpose(np.array(listeA)-np.mean(listeA,axis=0)),np.array(listeB)-np.mean(listeB,axis=0))
        U,S,V=np.linalg.svd(H)
        R=np.dot(V,np.transpose(U))
        t=np.mean(listeB,axis=0)-np.dot(R,np.mean(listeA,axis=0))
        print("number_common_nodes_found :",len(listeA))
        if np.linalg.norm(t)<=convergence_threshold:
            converged=True
        R0=np.dot(R,R0)
        t0=t+t0
    print("Find R and T, t=",tim-time())
    tim=time()
    skeleton_transformed=transform_skeleton(skeleton1,R0,t0)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    print("transform, dilate and thin, t=",tim-time())
    tim=time()
    if len(save)>=0:
        from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(save+'_raw_aligned_skeleton.csv')
        np.savetxt(save+'rot.txt',R0)
        np.savetxt(save+'trans.txt',t0)
    print("R0=",R0,'t0=',t0)
    return(skeleton_transformed)

def reconnect(skeleton):
    skeleton_transformed=dilate(skeleton)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    return(skeleton_transformed)


def shift(skeleton1,skeleton2):
    skeleton1_dilated = dilate(dilate(skeleton1)).astype(np.float)
    skeleton2_dilated = dilate(dilate(skeleton2)).astype(np.float)
    def distance(shift):
        distance=0
#         print(shift)
        for pixel in skeleton1_dilated.keys():
#             print(pixel[0]+shift[0],pixel[1]+shift[1])
            if (skeleton2_dilated.shape[0]>np.ceil(pixel[0]+shift[0])>=0 and skeleton2_dilated.shape[1]>np.ceil(pixel[1]+shift[1])>=0):
                shifted_pixel = (int(pixel[0]+shift[0]),int(pixel[1]+shift[1]))
                shifted_pixel_next = (np.ceil(pixel[0]+shift[0]),np.ceil(pixel[1]+shift[1]))
#                 print(shifted_pixel)
                prop=1/2*(pixel[0]+shift[0]-int(pixel[0]+shift[0])+pixel[1]+shift[1]-int(pixel[1]+shift[1]))
                float_value=(1-prop)*skeleton2_dilated[shifted_pixel[0],shifted_pixel[1]]+prop*(skeleton2_dilated[shifted_pixel_next[0],shifted_pixel_next[1]])
                distance+=abs(skeleton1_dilated[pixel]-float_value)
            else:
                distance+=1
#         for pixel in skeleton2_dilated.keys():
#             if (skeleton2_dilated.shape[0]>pixel[0]-shift[0]>=0 and skeleton2_dilated.shape[1]>pixel[1]-shift[1]>=0):
#                 shifted_pixel = (int(pixel[0]-shift[0]),int(pixel[1]-shift[1]))
#                 distance+=abs(skeleton1_dilated[shifted_pixel[0],shifted_pixel[1]]^skeleton2_dilated[pixel])
#             else:
#                 distance+=1
#         print(distance)
        return distance
    return(minimize(distance,np.array([10,10]), method='nelder-mead',options={'xatol': 1, 'disp': True,'fatol':0.1}))


def realign_final(skeleton1,skeleton2,t_init=np.array([0,0]),Rot_init=np.array([[1,0],[0,1]])):
    nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
    nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
    pruned1 = prune_graph(nx_graph1)
    pruned2 = prune_graph(nx_graph2)
    X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
    Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
    Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    ax.scatter(Xex[0,:],Xex[1,:])
    ax.scatter(Y[0,:],Y[1,:])
    X = np.insert(X, 2, values=0, axis=0) 
    Y = np.insert(Y, 2, values=0, axis=0) 
    print(X.shape,Y.shape)
    vectorX = o3d.utility.Vector3dVector(np.transpose(X))
    vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
    source = o3d.geometry.PointCloud(vectorX)
    target = o3d.geometry.PointCloud(vectorY)
    threshold = 200
    trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                             [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                             [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
    reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    Rfound = reg_p2p.transformation[0:2,0:2]
    tfound = reg_p2p.transformation[0:2,3]
    print(Rfound,tfound)
    X,Y=X[0:2,:],Y[0:2,:]
    Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
    ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    skel_transformed = transform_skeleton(skeleton1,Rfound,tfound)
    skel_mat = np.zeros((30000, 50000),dtype=np.uint8)
    for pixel in skel_transformed.keys():
        skel_mat[pixel]=1
    return(skel_mat,skel_transformed,Rfound,tfound)

def find_circle(x,y):
    x_m = np.mean(x)
    y_m = np.mean(y)

    # calculation of the reduced coordinates
    u = x - x_m
    v = y - y_m

    # linear system defining the center (uc, vc) in reduced coordinates:
    #    Suu * uc +  Suv * vc = (Suuu + Suvv)/2
    #    Suv * uc +  Svv * vc = (Suuv + Svvv)/2
    Suv  = np.sum(u*v)
    Suu  = np.sum(u**2)
    Svv  = np.sum(v**2)
    Suuv = np.sum(u**2 * v)
    Suvv = np.sum(u * v**2)
    Suuu = np.sum(u**3)
    Svvv = np.sum(v**3)

    # Solving the linear system
    A = np.array([ [ Suu, Suv ], [Suv, Svv]])
    B = np.array([ Suuu + Suvv, Svvv + Suuv ])/2.0
    uc, vc = np.linalg.solve(A, B)

    xc_1 = x_m + uc
    yc_1 = y_m + vc

    # Calcul des distances au centre (xc_1, yc_1)
    Ri_1     = np.sqrt((x-xc_1)**2 + (y-yc_1)**2)
    R_1      = np.mean(Ri_1)
    residu_1 = np.sum((Ri_1-R_1)**2)
    return(R_1,xc_1,yc_1)

In [28]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[12:14]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
graph_pos=[]
contours=[]
half_circles=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
#     contour = skel_info['contour']
#     half_circle = skel_info['half_circle']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    contours.append(contour)
    half_circles.append(half_circle)
#     nx_graph,pos = generate_nx_graph(from_sparse_to_graph(skel_doc))
#     graph_pos.append((nx_graph,pos))
skel_doc = skel_docs[0]
skel_aligned_t = skels[0].todense()
skel_sparse = scipy.sparse.csc_matrix(skels[0])
directory_name=f'2020{dates[0]}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
sio.savemat(path_snap+'/Analysis/skeleton_realigned.mat',{'skeleton' : skel_sparse,'R' : np.array([[1,0],[0,1]]),'t' : np.array([0,0])})
for i,skel in enumerate(skel_docs[1:]):
    directory_name=f'2020{dates[i+1]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    contour1,contour2 = contours[i],contours[i-1]
    half_circle1,half_circle2 = half_circles[i],half_circles[i-1]
    R1,xc1,yc1 = find_circle(*half_circle1.nonzero())
    R2,xc2,yc2 = find_circle(*half_circle2.nonzero())
    #prealignment based on half circle
    X = np.array(contour1.nonzero())
    Y = np.array(contour2.nonzero())
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    ax.scatter(X[0,:],X[1,:])
    ax.scatter(Y[0,:],Y[1,:])
    X = np.insert(X, 2, values=0, axis=0) 
    Y = np.insert(Y, 2, values=0, axis=0) 
    print(X.shape,Y.shape)
    vectorX = o3d.utility.Vector3dVector(np.transpose(X))
    vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
    source = o3d.geometry.PointCloud(vectorX)
    target = o3d.geometry.PointCloud(vectorY)
    threshold = 200
    t_init=np.array([ xc_1-xc_2, yc_1-yc_2])
    Rot_init = np.array([[ 1, 0],[ 0,  1]])
    trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                             [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                             [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])

    reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    Rfound = reg_p2p.transformation[0:2,0:2]
    tfound = reg_p2p.transformation[0:2,3]
    print(Rfound,tfound)
    X,Y=X[0:2,:],Y[0:2,:]
    Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
    skel_aligned,skel_doc,R,t = realign_final(skel,skel_doc)
    contours[i] = transform_skeleton(contours[i],R,t)
    half_circles[i] = transform_skeleton(half_circles[i],R,t)
    skel_sparse = scipy.sparse.csc_matrix(skel_aligned)
    sio.savemat(path_snap+'/Analysis/skeleton_realigned.mat',{'skeleton' : skel_sparse,'R' : R,'t' : t})
    plot_t_tp1([],[],None,None,skel_aligned[19000:21000,44000:46000],skel_aligned_t[19000:21000,44000:46000])
    skel_aligned_t = skel_aligned

In [30]:
skel_aligned,skel_doc,R,t = realign_final(skel_docs[0],skel_docs[1], t_init=np.array([1936,-2019]),Rot_init= np.array([[ 0.99999938, -0.00111566],
 [ 0.00111566,  0.99999938]]))



[4937.82736]
[10913.437609]


ValueError: operands could not be broadcast together with shapes (2,4908) (2,) 

In [40]:
plt.close('all')
# skeleton1,skeleton2 = skel_docs[0],skel_docs[1]
t_init=np.array([1936,-2019])
Rot_init= np.array([[ 0.99999938, -0.00111566],
 [ 0.00111566,  0.99999938]])
# nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
# nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
# pruned1 = prune_graph(nx_graph1)
# pruned2 = prune_graph(nx_graph2)
X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(Xex[0,:],Xex[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 20
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

skel_transformed = transform_skeleton(skeleton1,Rfound,tfound)
skel_mat = np.zeros((30000, 50000),dtype=np.uint8)
for pixel in skel_transformed.keys():
    skel_mat[pixel]=1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 4908) (3, 7337)
registration::RegistrationResult with fitness=8.088835e-02, inlier_rmse=1.386270e+01, and correspondence_set size of 397
Access transformation to get result.
[[ 9.99999533e-01 -9.68449308e-04]
 [ 9.68449308e-04  9.99999533e-01]] [ 1923.4725809 -2012.1459129]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
%matplotlib widget
plot_t_tp1([],[],None,None,skel_aligned[19000:21000,44000:46000],skels[1][19000:21000,44000:46000].todense())


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
skel_docs[1]

In [25]:
X=np.array(get_degree3_nodes(skel_docs[0]))
Y = np.array(get_degree3_nodes(skel_docs[1]))


In [28]:
X = np.transpose(X)
Y=np.transpose(Y)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 0.2
t_init=np.array([ 1774.73372876, -1144.44026333])
Rot_init = np.array([[ 9.99999951e-01, -3.15042571e-04],[ 3.15042571e-04,  9.99999951e-01]])
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])

reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 5219) (3, 7779)
registration::RegistrationResult with fitness=1.149646e-03, inlier_rmse=9.666348e-02, and correspondence_set size of 6
Access transformation to get result.
[[ 9.99999966e-01 -2.63180600e-04]
 [ 2.63180600e-04  9.99999966e-01]] [ 1772.54157915 -1143.34802684]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
for pixel in skel_doc.keys():
    skel_aligned[pixel]=1
plot_t_tp1([],[],None,None,skel_aligned[19000:21000,44000:46000],skel_aligned[19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
skels[1].todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [69]:
plt.close('all')

In [6]:
plot_t_tp1([],[],None,None,skels[1][20000:24000,31000:35000].todense(),skels[1][20000:24000,31000:35000].todense())


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
plot_t_tp1([],[],None,None,skels[0][15000:19000,38000:42000].todense(),skels[1][15000:19000,38000:42000].todense())


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
plt.close('all')
plot_t_tp1([],[],None,None,skel_aligned[19000:21000,44000:46000],skel_aligned_t[19000:21000,44000:46000])


NameError: name 'plt' is not defined

In [ ]:
    plot_t_tp1([],[],None,None,skel_aligned[19000:21000,44000:46000],skel_aligned_t[19000:21000,44000:46000])


In [74]:
R
trans_init

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [52]:
skels

[<26387x50121 sparse matrix of type '<class 'numpy.uint8'>'
 	with 1017225 stored elements in Compressed Sparse Column format>,
 <26217x49608 sparse matrix of type '<class 'numpy.uint8'>'
 	with 1031982 stored elements in Compressed Sparse Column format>]

In [8]:
import sys
sys.getsizeof(skels[0])

64

In [7]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[4:8]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels_aligned = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    dilated = read_mat(path_snap+'/Analysis/dilated.mat')['dilated']
    skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')['skeleton']
    skels_aligned.append(scipy.sparse.dok_matrix(skel))
nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(skeleton)) for skeleton in skels_aligned]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[clean_degree_4(prune_graph(nx_graph),poss_aligned[i])[0] for i,nx_graph in enumerate(nx_graphs_aligned)]

[1465.453887]
cleaning, number of nodes before 4217
number of unsolved cases 208
number of unsolved cases 110
number of unsolved cases 102
number of unsolved cases 102
end cleaning, number of nodes after 3493
[1564.79882]
cleaning, number of nodes before 4292
number of unsolved cases 173
number of unsolved cases 90
number of unsolved cases 83
number of unsolved cases 83
number of unsolved cases 83
number of unsolved cases 83
end cleaning, number of nodes after 3613
[2003.15984]
cleaning, number of nodes before 4840
number of unsolved cases 144
number of unsolved cases 59
number of unsolved cases 54
number of unsolved cases 54
end cleaning, number of nodes after 4006
[2568.74304]
cleaning, number of nodes before 5520
number of unsolved cases 276
number of unsolved cases 114
number of unsolved cases 98
number of unsolved cases 89
number of unsolved cases 87
number of unsolved cases 83
number of unsolved cases 83
end cleaning, number of nodes after 4535


In [10]:
import pickle
for i,g in enumerate(nx_graph_pruned):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    pos = poss_aligned[i]
    pickle.dump((g,pos),open( path_save, "wb" ))
    

In [11]:
nx_graph_pos_retrieved=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    (g,pos) = pickle.load(open( path_save, "rb" ))
    nx_graph_pos_retrieved.append((g,pos))

In [40]:
plot_t_tp1([],[],None,None,skel_aligned_dense[1][19000:21000,44000:46000],skel_aligned_dense[2][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
skels_aligned[1]

In [6]:
scipy.sparse.csc_matrix(skel)

<26316x49503 sparse matrix of type '<class 'numpy.uint8'>'
	with 465757 stored elements in Compressed Sparse Column format>

In [3]:
pos_t = graph_pos[0][1]
pos_tp1 = graph_pos[1][1]
pruned=[]

X = np.transpose(np.array([pos_t[node] for node in pruned[0] if pruned[0].degree(node)==3]))
Y = np.transpose(np.array([pos_tp1[node] for node in pruned[1] if pruned[1].degree(node)==3]))
# X=[]
# Y=[]
# num=5
# nodesX=[choice([node for node in pruned[0] if pruned[0].degree(node)==3]) for i in range(num)]
# nodesY = [choice([node for node in pruned[1] if pruned[1].degree(node)==3]) for i in range(num)]
# window = 200
# g = pruned[0]
# for node in nodesX:
#     pos = pos_t[node]
#     xy = skels[0][max(0,pos[0]-window):pos[0]+window,max(0,pos[1]-window):pos[1]+window].nonzero()
#     xy2 = skels[1][max(0,pos[0]-window):pos[0]+window,max(0,pos[1]-window):pos[1]+window].nonzero()
#     for i,x in enumerate(xy[0]):
#         y = xy[1][i]
#         X.append((pos[0]-window+x,pos[1]-window+y))
#         if i<len(xy2[0]):
#             x2,y2 = xy2[0][i],xy2[1][i]
#             Y.append((pos[0]-window+x2,pos[1]-window+y2))
# X=np.transpose(np.array(X))
# Y=np.transpose(np.array(Y))
X.shape,Y.shape
fig=plt.figure(figsize=(10,9))

ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 20
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 2763) (3, 3131)
registration::RegistrationResult with fitness=9.290626e-01, inlier_rmse=5.239506e+00, and correspondence_set size of 2567
Access transformation to get result.
[[ 9.99999893e-01 -4.63437568e-04]
 [ 4.63437568e-04  9.99999893e-01]] [ 19.51041473 -32.80502458]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
np.linalg.inv(Rfound)

array([[ 9.99999893e-01,  4.63437568e-04],
       [-4.63437568e-04,  9.99999893e-01]])

In [7]:
skel_transformed = transform_skeleton(skel_docs[0],Rfound,tfound)

In [7]:
def prune_graph(nx_graph,threshold=100):
    #should implement threshold!
    S = [nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph)]
    selected = [g for g in S if g.size(weight="weight")*len(g.nodes)/10**6>=threshold]
    len_connected=[(nx_graph.size(weight="weight")*len(nx_graph.nodes)/10**6) for nx_graph in selected]
    print(len_connected)
    G = selected[0]
    for g in selected[1:]:
        G = nx.compose(G,g)
    return(G)
def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = np.round(np.transpose(np.dot(Rot,np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int)
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)

In [27]:
def realign_final(skeleton1,skeleton2):
    nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
    nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
    pruned1 = prune_graph(nx_graph1)
    pruned2 = prune_graph(nx_graph2)
    X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
    Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
#     fig=plt.figure(figsize=(10,9))

#     ax = fig.add_subplot(111)
#     ax.scatter(X[0,:],X[1,:])
#     ax.scatter(Y[0,:],Y[1,:])
    X = np.insert(X, 2, values=0, axis=0) 
    Y = np.insert(Y, 2, values=0, axis=0) 
    print(X.shape,Y.shape)
    vectorX = o3d.utility.Vector3dVector(np.transpose(X))
    vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
    source = o3d.geometry.PointCloud(vectorX)
    target = o3d.geometry.PointCloud(vectorY)
    threshold = 20
    trans_init = np.asarray([[1, 0, 0, 0],
                             [0, 1, 0, 0],
                             [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
    reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    Rfound = reg_p2p.transformation[0:2,0:2]
    tfound = reg_p2p.transformation[0:2,3]
    print(Rfound,tfound)
    X,Y=X[0:2,:],Y[0:2,:]
    Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
#     fig=plt.figure(figsize=(10,9))
#     ax = fig.add_subplot(111)
#     ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
#     ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    skel_transformed = transform_skeleton(skeleton1,Rfound,tfound)
    skel_mat = np.zeros((26322, 49527),dtype=np.uint8)
    for pixel in skel_transformed.keys():
        skel_mat[pixel]=1
    return(skel_mat,skel_doc,Rfound,tfound)

In [15]:
final_realigned = realign_final(skel_docs[1],skel_docs[0])

[1993.99993]
[1581.43881]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 3131) (3, 2763)
registration::RegistrationResult with fitness=8.272118e-01, inlier_rmse=5.422845e+00, and correspondence_set size of 2590
Access transformation to get result.
[[ 9.99999904e-01  4.39147129e-04]
 [-4.39147129e-04  9.99999904e-01]] [-18.47679909  32.34018681]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
skel_mat = np.zeros(skels[0].shape,dtype=np.uint8)
for pixel in skel_transformed.keys():
    skel_mat[pixel]=1
# plt.close('all')
plot_t_tp1([],[],None,None,final_realigned[0][19000:21000,44000:46000],skels[0][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
def get_neighbours(pixel,non_zero_pixel):
    x=pixel[0]
    y=pixel[1]
    primary_neighbours = {(x+1,y),(x-1,y),(x,y+1),(x,y-1)}
    secondary_neighbours = {(x+1,y-1),(x+1,y+1),(x-1,y+1),(x-1,y-1)}
    num_neighbours = 0
    actual_neighbours = []
    for neighbour in primary_neighbours:
        if neighbour in non_zero_pixel:
            num_neighbours +=1
            xp=neighbour[0]
            yp=neighbour[1]
            primary_neighboursp = {(xp+1,yp),(xp-1,yp),(xp,yp+1),(xp,yp-1)}
            for neighbourp in primary_neighboursp:
                secondary_neighbours.discard(neighbourp)
            actual_neighbours.append(neighbour)
    for neighbour in secondary_neighbours:
        if neighbour in non_zero_pixel:
            num_neighbours +=1
            actual_neighbours.append(neighbour)
    return(actual_neighbours,num_neighbours)
def get_degree3_nodes(skel):
    deg_3=[]
    non_zero= skel.keys()
    for pixel in non_zero:
        n, num = get_neighbours(pixel,non_zero)
        if num ==3:
            deg_3.append(pixel)
    return(deg_3)

In [20]:
def get_degree3_nodes(skel):
    deg_3=[]
    non_zero= skel.keys()
    for pixel in non_zero:
        n, num = get_neighbours(pixel,non_zero)
        if num ==3:
            deg_3.append(pixel)
    return(deg_3)

In [3]:
pos_t = graph_pos[0][1]
pos_tp1 = graph_pos[1][1]


In [4]:
pruned=[]
for nx_graph,pos in graph_pos:
    S = [nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph)]
    selected = [g for g in S if g.size(weight="weight")*len(g.nodes)/10**6>=100]
    pruned.append(selected[0])

In [34]:
pos[1]-window

38720

In [51]:
len(X),len(Y)

(4335, 3940)

In [38]:
X = np.transpose(np.array([pos_t[node] for node in pruned[0] if pruned[0].degree(node)==3 and get_importance(pruned[0],node)>=10]))
Y = np.transpose(np.array([pos_tp1[node] for node in pruned[1] if pruned[1].degree(node)==3 and get_importance(pruned[1],node)>=10]))
X.shape,Y.shape

NameError: name 'get_importance' is not defined

In [9]:
X = np.transpose(np.array([pos_t[node] for node in pruned[0] if pruned[0].degree(node)==3]))
Y = np.transpose(np.array([pos_tp1[node] for node in pruned[1] if pruned[1].degree(node)==3]))
X.shape,Y.shape

((2, 2763), (2, 3131))

In [78]:
def get_importance(g,node):
    score = np.inf
    for edge in g.edges(node):
        score =  min(score,g.get_edge_data(*edge)['weight'])
    return(score)

In [80]:
importance = []
g=pruned[0]
for node in g:
    importance.append(get_importance(g,node))

In [ ]:
fig=plt.figure(figsize=(14,11))
ax = fig.add_subplot(111)
ax.hist(importance,10)

In [4]:
num=1
nodesX=[choice([node for node in pruned[0] if pruned[0].degree(node)==3]) for i in range(num)]
nodesY = [choice([node for node in pruned[1] if pruned[1].degree(node)==3]) for i in range(num)]

In [5]:
X=[]
Y=[]
window = 200
g = pruned[0]
for node in nodesX:
    pos = pos_t[node]
    xy = skels[0][max(0,pos[0]-window):pos[0]+window,max(0,pos[1]-window):pos[1]+window].nonzero()
    xy2 = skels[1][max(0,pos[0]-window):pos[0]+window,max(0,pos[1]-window):pos[1]+window].nonzero()
    for i,x in enumerate(xy[0]):
        y = xy[1][i]
        X.append((pos[0]-window+x,pos[1]-window+y))
        if i<len(xy2[0]):
            x2,y2 = xy2[0][i],xy2[1][i]
            Y.append((pos[0]-window+x2,pos[1]-window+y2))


NameError: name 'pos_t' is not defined

In [10]:
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
X.shape,Y.shape


((3, 2763), (3, 3131))

In [4]:
X = []
dimx = 1000
dimy = 1000
for i in range(dimx):
    X.append((i,0))
    X.append((i,dimy))
for i in range(dimy):
    X.append((dimx,i))
    X.append((0,i))
X=np.transpose(np.array(X))
angle = np.random.uniform(0,np.pi/16)
R=np.array([[np.cos(angle),-np.sin(angle)],[np.sin(angle),np.cos(angle)]])
t = np.random.uniform(0,1,(2,1))
print(R,t)
Y=np.transpose(np.transpose(np.dot(R,X))+np.transpose(t))
np.random.shuffle(np.transpose(Y))
fig=plt.figure(figsize=(10,9))

ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 20
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

[[ 0.9966081 -0.0822939]
 [ 0.0822939  0.9966081]] [[0.11547774]
 [0.37731306]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=9.583941e-01, and correspondence_set size of 4000
Access transformation to get result.
Transformation is:
[[ 0.9968446  -0.07937788  0.         -1.40800181]
 [ 0.07937788  0.9968446   0.          1.7217043 ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [92]:
X.shape,Y.shape

((2, 2014), (2, 2014))

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
reg = RigidRegistration(R=np.array([[1,0],[0,1]]),t=np.array([[0,0]]),**{'X': np.transpose(X), 'Y': np.transpose(Y)})
out = reg.register()
Rfound = reg.R
tfound= reg.t
print(Rfound,tfound)
Yrep=np.transpose(np.transpose(np.dot(Rfound,np.transpose(np.transpose(X))))-tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
def realign_final(skeleton1,skeleton2):
    nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
    nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
    pruned1 = prune_graph(nx_graph1)
    pruned2 = prune_graph(nx_graph2)
    X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
    Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
    reg = RigidRegistration(R=np.array([[1,0],[0,1]]),t=np.array([[0,0]]),**{'X': np.transpose(X), 'Y': np.transpose(Y)})
    out = reg.register()
    Rfound = reg.R
    tfound= reg.t
    skel_transformed = transform_skeleton(skeleton2,Rfound,tfound)
    skel_mat = np.zeros((26322, 49527),dtype=np.uint8)
    for pixel in skel_transformed.keys():
        skel_mat[pixel]=1
    return(skel_mat,Rfound,tfound)

In [104]:

def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = (np.round(np.transpose(np.dot(np.linalg.inv(Rot),np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int))
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)
def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = (np.round(np.transpose(np.dot(np.linalg.inv(Rot),np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int))
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)
skel_mat3 = realign_final(skel_docs[0],skel_docs[1])

[700.88858]
[1012.7436]


In [122]:
skel_mat = np.zeros(skels[0].shape,dtype=np.uint8)
for pixel in skel_transformed.keys():
    skel_mat[pixel]=1

In [123]:
plt.close('all')
plot_t_tp1([],[],None,None,skel_mat[19000:21000,44000:46000],skels[0][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
plt.close('all')
plot_t_tp1([],[],None,None,skel_mat[19000:21000,44000:46000],skels[1][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
plot_t_tp1([],[],None,None,skel_mat2[19000:21000,44000:46000],skels[1][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plot_t_tp1([],[],None,None,skels[0][19000:21000,44000:46000],skels[1][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
import pandas as pd
import networkx as nx
import numpy as np
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_nx_to_tab
from node_id import whole_movement_identification
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from sparse_util import dilate, zhangSuen
from scipy.optimize import minimize
from time import time


def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = np.round(np.transpose(np.dot(Rot,np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int)
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)

def realign(skeleton1,nx_graphB,posB,convergence_threshold,window=500,maxdist=50,save=''):
    converged=False
    nx_graphA,posA=generate_nx_graph_from_skeleton(skeleton1) 
    t0=np.array([0,0])
    R0=np.identity(2)
    while not converged:
        listeA,listeB = find_common_group_nodes(nx_graphA,nx_graphB,posA,posB,R0,t0,maxdist=maxdist,window=window)
        H=np.dot(np.transpose(np.array(listeA)-np.mean(listeA,axis=0)),np.array(listeB)-np.mean(listeB,axis=0))
        U,S,V=np.linalg.svd(H)
        R=np.dot(V,np.transpose(U))
        t=np.mean(listeB,axis=0)-np.dot(R,np.mean(listeA,axis=0))
        print("number_common_nodes_found :",len(listeA))
        if np.linalg.norm(t)<=convergence_threshold:
            converged=True
        R0=np.dot(R,R0)
        t0=t+t0
    skeleton_transformed=transform_skeleton(skeleton1,R0,t0)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    if len(save)>=0:
        from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(save+'_raw_aligned_skeleton.csv')
        np.savetxt(save+'rot.txt',R0)
        np.savetxt(save+'trans.txt',t0)
    print("R0=",R0,'t0=',t0)
    return(skeleton_transformed)

def find_common_group_nodes(Sa,Sb,degree3_nodesa,degree3_nodesb,posa,posb,R0,t0,window=500,maxdist=50):
    common_nodes_a = []
    common_nodes_b = []
    common_centroida = []
    common_centroidb = []
    t=time()
    posarottrans = {key : np.round(np.transpose(np.dot(R0,np.transpose(np.array(posa[key])))+t0)).astype(np.int) for key in degree3_nodesa}
#     print("rotating translating",time()-t)
    for node in degree3_nodesa:
        t=time()
        posanchor=posarottrans[node]
        potential_surroundinga=Sa[posanchor[0]-2*window:posanchor[0]+2*window,posanchor[1]-2*window:posanchor[1]+2*window]
        potential_surroundingb=Sb[posanchor[0]-2*window:posanchor[0]+2*window,posanchor[1]-2*window:posanchor[1]+2*window]
#         print("candidates",len(potential_surroundinga.data))
#         print("finding_potential_surrounding",time()-t)
        t=time()
        surrounding_nodesa=[node for node in potential_surroundinga.data if 
                            (posanchor[0]-window<posarottrans[int(node)][0]<posanchor[0]+window and posanchor[1]-window<posarottrans[int(node)][1]<posanchor[1]+window 
                             )]
        surrounding_nodesb=[node for node in potential_surroundingb.data if 
                    (posanchor[0]-window<posb[int(node)][0]<posanchor[0]+window and posanchor[1]-window<posb[int(node)][1]<posanchor[1]+window 
                    )]
#         print("finding_surrounding",time()-t)
        t=time()
        if len(surrounding_nodesa)==len(surrounding_nodesb):
            possurroundinga=[posarottrans[node] for node in surrounding_nodesa]
            possurroundingb=[posb[node] for node in surrounding_nodesb]
            centroida= np.mean(possurroundinga,axis=0)
            centroidb= np.mean(possurroundingb,axis=0)
            if np.linalg.norm(centroida-centroidb)<=maxdist:
                common_centroida.append(centroida)
                common_centroidb.append(centroidb)
    return(common_centroida,common_centroidb)

def realign2(skeleton1,skeleton2,convergence_threshold,window=500,maxdist=50,save=''):
    converged=False
    tim=time()
    nx_graphA,posA=generate_nx_graph_from_skeleton(skeleton1)
    nx_graphB,posB=generate_nx_graph_from_skeleton(skeleton2)
    print("generate_nx_graph_from_skeleton, t=",tim-time())
    tim=time()
    t0=np.array([0,0])
    R0=np.identity(2)
    degree3_nodesa = [node for node in nx_graphA if nx_graphA.degree(node)==3]
    degree3_nodesb = [node for node in nx_graphB if nx_graphB.degree(node)==3]
    print("lennodes=",len(degree3_nodesa))
    Sa=sparse.csr_matrix((26296, 49559))
    Sb=sparse.csr_matrix((26296, 49559))
    for node in degree3_nodesa:
        Sa[posA[node][0],posA[node][1]]=node
    for node in degree3_nodesb:
        Sb[posB[node][0],posB[node][1]]=node
    while not converged:
        listeA,listeB = find_common_group_nodes(Sa,Sb,degree3_nodesa,degree3_nodesb,posA,posB,R0,t0,maxdist=maxdist,window=window)
        H=np.dot(np.transpose(np.array(listeA)-np.mean(listeA,axis=0)),np.array(listeB)-np.mean(listeB,axis=0))
        U,S,V=np.linalg.svd(H)
        R=np.dot(V,np.transpose(U))
        t=np.mean(listeB,axis=0)-np.dot(R,np.mean(listeA,axis=0))
        print("number_common_nodes_found :",len(listeA))
        if np.linalg.norm(t)<=convergence_threshold:
            converged=True
        R0=np.dot(R,R0)
        t0=t+t0
    print("Find R and T, t=",tim-time())
    tim=time()
    skeleton_transformed=transform_skeleton(skeleton1,R0,t0)
    skeleton_transformed=dilate(skeleton_transformed)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    print("transform, dilate and thin, t=",tim-time())
    tim=time()
    if len(save)>=0:
        from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(save+'_raw_aligned_skeleton.csv')
        np.savetxt(save+'rot.txt',R0)
        np.savetxt(save+'trans.txt',t0)
    print("R0=",R0,'t0=',t0)
    return(skeleton_transformed)

def reconnect(skeleton):
    skeleton_transformed=dilate(skeleton)
    skeleton_transformed=zhangSuen(skeleton_transformed)
    return(skeleton_transformed)


def shift(skeleton1,skeleton2):
    skeleton1_dilated = dilate(dilate(skeleton1)).astype(np.float)
    skeleton2_dilated = dilate(dilate(skeleton2)).astype(np.float)
    def distance(shift):
        distance=0
#         print(shift)
        for pixel in skeleton1_dilated.keys():
#             print(pixel[0]+shift[0],pixel[1]+shift[1])
            if (skeleton2_dilated.shape[0]>np.ceil(pixel[0]+shift[0])>=0 and skeleton2_dilated.shape[1]>np.ceil(pixel[1]+shift[1])>=0):
                shifted_pixel = (int(pixel[0]+shift[0]),int(pixel[1]+shift[1]))
                shifted_pixel_next = (np.ceil(pixel[0]+shift[0]),np.ceil(pixel[1]+shift[1]))
#                 print(shifted_pixel)
                prop=1/2*(pixel[0]+shift[0]-int(pixel[0]+shift[0])+pixel[1]+shift[1]-int(pixel[1]+shift[1]))
                float_value=(1-prop)*skeleton2_dilated[shifted_pixel[0],shifted_pixel[1]]+prop*(skeleton2_dilated[shifted_pixel_next[0],shifted_pixel_next[1]])
                distance+=abs(skeleton1_dilated[pixel]-float_value)
            else:
                distance+=1
#         for pixel in skeleton2_dilated.keys():
#             if (skeleton2_dilated.shape[0]>pixel[0]-shift[0]>=0 and skeleton2_dilated.shape[1]>pixel[1]-shift[1]>=0):
#                 shifted_pixel = (int(pixel[0]-shift[0]),int(pixel[1]-shift[1]))
#                 distance+=abs(skeleton1_dilated[shifted_pixel[0],shifted_pixel[1]]^skeleton2_dilated[pixel])
#             else:
#                 distance+=1
#         print(distance)
        return distance
    return(minimize(distance,np.array([10,10]), method='nelder-mead',options={'xatol': 1, 'disp': True,'fatol':0.1}))